## Hyperparameter Tuning

The hyperparameters in deep learning to tune are 
- the number of neurons
- activation function
- optimiser
- learning rate
- batch size
- epochs 
- number of layers.


Reference: 
- https://www.analyticsvidhya.com/blog/2021/05/tuning-the-hyperparameters-and-layers-of-neural-network-deep-learning/

In [53]:
from typing import Tuple, List, Optional, Callable
import random
import os
from tqdm import tqdm
import gdown
import zipfile

import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset, random_split
from torchvision.models import efficientnet_b5, EfficientNet_B5_Weights

from ray import tune
from ray.tune.search.optuna import OptunaSearch
from ray.tune.schedulers import ASHAScheduler

print("Libraries imported. Using device:", "cuda" if torch.cuda.is_available() else "cpu")

Libraries imported. Using device: cpu


In [54]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import torch.nn as nn
import torch
import torch.optim as optim
from transformers import AutoImageProcessor, Dinov2ForImageClassification

from PIL import Image
import random
import os

from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
import seaborn as sns


## Data Download
This is adapted from SeparatingData.ipynb
Download the processed dataset from Google Drive is yet to.

In [55]:
def download_dataset(data_dir: str, zip_url: str, zip_filename: str, root_dir: str) -> None:
    """
    Download and extract the dataset from Google Drive if it doesn't exist

    Args:
        data_dir (str): Directory where the dataset zip file will be stored
        zip_url (str): URL of the dataset zip file
        zip_filename (str): Name for the downloaded zip file
        root_dir (str): Directory where the dataset will be extracted
    """
    # Create the data directory if it doesn't exist.
    if not os.path.exists(root_dir):
        os.makedirs(root_dir)
        print(f"Created directory: {root_dir}")

    # Check if the dataset is already extracted.
    if not os.path.exists(data_dir):
        print(f"Downloading dataset from {zip_url} to {zip_filename}")
        gdown.download(zip_url, zip_filename, quiet=False)
        print("Extracting dataset...")
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall(root_dir)
        print(f"Extraction complete. Dataset available at {data_dir}")
    else:
        print(f"Dataset already exists at {data_dir}")

In [56]:
DATA_DIR = "../input/final_split.zip"
ZIP_URL = "https://drive.google.com/uc?id=11t8m703wcNss3w5diJSUGBA_vXnCKChr"
ZIP_FILENAME = "../input/final_split.zip"
ROOT_DIR = "../input"

download_dataset(DATA_DIR, ZIP_URL, ZIP_FILENAME, ROOT_DIR)

Dataset already exists at ../input/final_split.zip


## Subset Data Load
For fasting hyperparameter tuning, use a subset of the dataset to find the most optimised set of hyperparameters.
Loads dataset from processed dataset which should have been split to train, test, eval.

In [57]:
def create_tuning_data_loaders(
    dataset_root: str,
    transform: transforms.Compose,
    batch_size: int = 32,
    subset_fraction: float = 0.1,
    random_seed: int = 42
) -> Tuple[DataLoader, DataLoader, DataLoader]:
    """Creates DataLoaders for hyperparameter tuning using a subset of the dataset.

    If the dataset_root directory contains subfolders 'train', 'eval', and 'test',
    these are loaded directly. Otherwise, the full dataset is loaded and randomly split.
    In either case, a subset of each split is sampled for faster tuning.

    Note:
      The transform provided is applied to each image as it is loaded by ImageFolder.
      Even if the images in the pre-split folders have been preprocessed externally,
      it is common to store raw images and apply transforms on the fly.

    Args:
        dataset_root (str): Root directory of the dataset. This should either be the parent
            folder of the split directories or the folder containing all images.
        transform (transforms.Compose): Transformations to apply to the dataset.
        batch_size (int, optional): Batch size for DataLoaders. Defaults to 32.
        subset_fraction (float, optional): Fraction of each split to use for tuning. Defaults to 0.1.
        random_seed (int, optional): Random seed for reproducibility. Defaults to 42.

    Returns:
        Tuple[DataLoader, DataLoader, DataLoader]: DataLoaders for training, validation, and test subsets.
    """
    # Check if dataset_root contains pre-split 'train', 'eval', and 'test' subdir
    split_dirs = ['train', 'eval', 'test']
    if all(os.path.exists(os.path.join(dataset_root, sub)) for sub in split_dirs):
        train_dataset = datasets.ImageFolder(root=os.path.join(dataset_root, 'train'), transform=transform)
        print(f"Full train dataset: {len(train_dataset)}")
        val_dataset = datasets.ImageFolder(root=os.path.join(dataset_root, 'eval'), transform=transform)
        print(f"Full val dataset: {len(val_dataset)}")
        test_dataset = datasets.ImageFolder(root=os.path.join(dataset_root, 'test'), transform=transform)
        print(f"Full test dataset: {len(test_dataset)}")

    else:
        # If not pre-split, load the full dataset and split it randomly
        full_dataset = datasets.ImageFolder(root=dataset_root, transform=transform)
        total_len = len(full_dataset)
        train_len = int(0.7 * total_len)
        val_len = int(0.15 * total_len)
        test_len = total_len - train_len - val_len
        train_dataset, val_dataset, test_dataset = random_split(
            full_dataset, [train_len, val_len, test_len],
            generator=torch.Generator().manual_seed(random_seed)
        )

    def sample_subset(dataset, fraction):
        """Returns a subset of the dataset with the specified fraction of examples."""
        dataset_len = len(dataset)
        subset_size = max(1, int(fraction * dataset_len))
        indices = random.sample(range(dataset_len), subset_size)
        return Subset(dataset, indices)

    # Sample a subset from each split
    if subset_fraction < 1.0:
        subset_train_dataset = sample_subset(train_dataset, subset_fraction)
        print(f"Subset-train dataset: {len(train_dataset)}")
        subset_val_dataset = sample_subset(val_dataset, subset_fraction)
        print(f"Subset-val dataset: {len(val_dataset)}")
        subset_test_dataset = sample_subset(test_dataset, subset_fraction)
        print(f"Subset-test dataset: {len(test_dataset)}")

    subset_train_loader = DataLoader(subset_train_dataset, batch_size=batch_size, shuffle=True)
    subset_val_loader = DataLoader(subset_val_dataset, batch_size=batch_size, shuffle=False)
    subset_test_loader = DataLoader(subset_test_dataset, batch_size=batch_size, shuffle=False)

    return subset_train_loader, subset_val_loader, subset_test_loader

In [58]:
SPLIT_DATASET = os.path.abspath("../input/final_split_training_augmented")
BATCH_SIZE = 64

# Define a transform. 
# In this notebook, we assume we doing it for EfficientNetB5, so we resize to ~456x456
weights = EfficientNet_B5_Weights.DEFAULT
transform = transforms.Compose([
        transforms.Resize((456, 456)),
        transforms.ToTensor(),
        weights.transforms()  # applies normalization as required
    ])

TRAIN_LOADER, VAL_LOADER, TEST_LOADER = create_tuning_data_loaders(
    dataset_root=SPLIT_DATASET,
    transform=transform,
    batch_size=BATCH_SIZE,
    subset_fraction=0.1,
    random_seed=42
)

print("DataLoaders for hyperparameter tuning are ready.")

Full train dataset: 8025
Full val dataset: 579
Full test dataset: 572
Subset-train dataset: 8025
Subset-val dataset: 579
Subset-test dataset: 572
DataLoaders for hyperparameter tuning are ready.


## Model Specifications
This is where you should replace with your model.

EfficientNetB5 Partial Transfer Learning:
- https://discuss.pytorch.org/t/partial-transfer-learning-efficientnet/109689 

In [59]:
rootPath = "../input/final_split_training_augmented"
trainPath= rootPath + "/train"
evalPath= rootPath + "/eval"
testPath= rootPath + "/test"

model_name = "facebook/dinov2-small-imagenet1k-1-layer"
num_classes = 4
batch_size = 32
learning_rate = 5e-5
epochs = 5
reduction_ratio = 0.2  # Define the reduction ratio for the dataset


# For MacOS MPS
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
torch.backends.mps.is_available()
print("Using device:", "mps" if torch.backends.mps.is_available() else "cpu")

# For CUDA
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print("Using device:", "cuda" if torch.cuda.is_available() else "cpu")

Using device: mps


In [60]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

def create_reduced_dataset(full_dataset, reduction_ratio):
    total_size = len(full_dataset)
    reduced_size = int(total_size * reduction_ratio)
    indices = list(range(total_size))
    random.shuffle(indices)
    reduced_indices = indices[:reduced_size]
    reduced_dataset = Subset(full_dataset, reduced_indices)
    print(f"Reduced {full_dataset} size: {len(reduced_dataset)}")
    return reduced_dataset

trainDataset = datasets.ImageFolder(root=trainPath, transform=transform)
evalDataset = datasets.ImageFolder(root=evalPath, transform=transform)

# Commenting out the reduced dataset creation for full training (or change above reduction ratio to 1.0)
trainDataset = create_reduced_dataset(trainDataset, reduction_ratio)
evalDataset = create_reduced_dataset(evalDataset, reduction_ratio)


train_dataloader = DataLoader(trainDataset, batch_size=batch_size, shuffle=True, num_workers=num_classes)
eval_dataloader = DataLoader(evalDataset, batch_size=batch_size, shuffle=False, num_workers=num_classes)


type(trainDataset)
first_element = trainDataset[0]
print(f"Type of the first element: {type(first_element)}")
image, label = first_element
print(f"Type of the image in the first element: {type(image)}")
print(f"Is the image a PIL Image? {isinstance(image, Image.Image)}")


Reduced Dataset ImageFolder
    Number of datapoints: 8025
    Root location: ../input/final_split_training_augmented/train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           ) size: 1605
Reduced Dataset ImageFolder
    Number of datapoints: 579
    Root location: ../input/final_split_training_augmented/eval
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           ) size: 115
Type of the first element: <class 'tuple'>
Type of the image in the first element: <class 'torch.Tensor'>
Is the image a PIL Image? False


In [61]:

class BaseDinov2(nn.Module):
    def __init__(
            self, model_name: str,
            num_classes: int,
            learning_rate: float, 
            freeze_backbone: bool = True):
        
        super().__init__()
        self.model_name = model_name
        self.num_classes = num_classes
        self.learning_rate = learning_rate
        self.freeze_backbone = freeze_backbone

        self.image_processor = AutoImageProcessor.from_pretrained(self.model_name)
        self.dinov2 = Dinov2ForImageClassification.from_pretrained(self.model_name, num_labels=self.num_classes, ignore_mismatched_sizes=True)

        if self.freeze_backbone:
            for name, param in self.dinov2.named_parameters():
                if "classifier" not in name:
                    param.requires_grad = False
                else:
                    param.requires_grad = True

        self.optimizer = optim.AdamW(self.dinov2.parameters(), lr=self.learning_rate)
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, pixel_values, labels=None):
        outputs = self.dinov2(pixel_values=pixel_values)
        logits = outputs.logits
        loss = self.criterion(logits, labels) if labels is not None else None
        return {"loss": loss, "logits": logits}

    def configure_optimizers(self):
        return self.optimizer

    def get_image_processor(self):
        return self.image_processor

In [62]:
model = BaseDinov2(model_name=model_name, num_classes=4, learning_rate=learning_rate, freeze_backbone=True)
print("Model instantiated:", model.__class__.__name__)

Some weights of Dinov2ForImageClassification were not initialized from the model checkpoint at facebook/dinov2-small-imagenet1k-1-layer and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model instantiated: BaseDinov2


## Hyperparameter Tuning
This is the part where you write the training function and load it to the ray tune scheduler.
For this execution, ASHAscheduler is used with Optuna for bayesian optimisation techniques - which should be using the default Tree-Structured Parzen Estimator.
If many parameters, this is would be more efficient than grid search and random search.

References:
- https://docs.ray.io/en/latest/tune/examples/includes/async_hyperband_example.html
- https://docs.ray.io/en/latest/tune/examples/tune-pytorch-cifar.html 
- https://docs.ray.io/en/latest/tune/examples/includes/mnist_pytorch.html
- https://docs.ray.io/en/latest/tune/api/suggestion.html

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from ray import tune

CHECKPOINT_DIR = os.path.abspath("../models/Dinov2_hyptune")
def train_model(config, checkpoint_dir=CHECKPOINT_DIR, data_dir=None):
    """Training function for Ray Tune hyperparameter tuning.

    This function instantiates the model with hyperparameters
    specified in the config dictionary, trains the model on the global TRAIN_LOADER,
    evaluates on VAL_LOADER, and reports the validation loss to Ray Tune.

    Args:
        config (dict): Hyperparameter configuration. Expected keys include:
            - lr (float): Learning rate.
            - weight_decay (float): Weight decay for the optimizer.
            - dropout (float): Dropout rate for the classifier.
            - hidden_sizes (list or None): List of hidden layer sizes in the classifier.
            - activation (str): Activation function to use ('relu', 'tanh', etc.).
            - freeze_backbone (bool): Whether to freeze the model backbone.
            - num_epochs (int): Number of training epochs.
            - optimiser (callable, optional): Optimiser class. Default is optim.Adam.
            - criterion (callable, optional): Loss function instance. Default is nn.CrossEntropyLoss().
        checkpoint_dir (str, optional): Directory for checkpointing (if applicable).
        data_dir (str, optional): Not used here; included for compatibility.
    """

    if checkpoint_dir:
        os.makedirs(checkpoint_dir, exist_ok=True)
        print(f"Checkpoint Folder exists")
    
    # instantiate model with hyperparameters from config
    model = BaseDinov2(
        num_classes=num_classes,
        # learning_rate=config.get("lr", learning_rate),
        # weight_decay=config.get("weight_decay", None),
        # dropout=config.get("dropout", None),
        # hidden_sizes=config.get("hidden_sizes", None),
        # activation=config.get("activation", None),
        freeze_backbone=config.get("freeze_backbone", True),
    ).to(device)
    
    optimizer_class = config.get("optimiser", optim.Adam)
    optimizer = optimizer_class(model.parameters(), lr=config["lr"], weight_decay=config["weight_decay"])
    
    criterion=config.get("criterion", nn.CrossEntropyLoss())
    num_epochs=config.get("num_epochs", 2)  # a low number for quick tuning, but update accordingly
    
    # image_processor = AutoImageProcessor.from_pretrained(model_name)
    # model = Dinov2ForImageClassification.from_pretrained(model_name, num_labels=num_classes, ignore_mismatched_sizes=True)

    train_losses = []
    train_accuracies = []
    eval_losses = []
    eval_accuracies = []
    all_true_labels = []
    all_predicted_labels = []

    for epoch in range(num_epochs):
        
        # Training loop
        model.train()
        total_loss = 0.0
        correct_predictions = 0
        total_samples = 0
        progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs}", unit="batch")
        # batch: In each iteration of this loop, batch will contain one batch of data loaded by your train_dataloader. Since you created the train_dataloader from a torch.utils.data.Dataset (specifically a Subset of ImageFolder), each batch will typically be a list or tuple containing:
        # A tensor of images (the first element, batch[0]). -> PyTorch tensors of shape (batch_size, C, H, W) e.g. (32, 3, 224, 224) for a batch size of 32 and colour images resized to 224x224.
        # A tensor of corresponding labels (the second element, batch[1]).
        for batch in progress_bar:

            inputs = BaseDinov2.image_processor(images=list(batch[0]), return_tensors="pt", do_rescale=False).to(device)
            labels = batch[1].to(device)

            optimizer.zero_grad()
            outputs = model(**inputs).logits
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * inputs['pixel_values'].size(0)
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

            progress_bar.set_postfix({"loss": total_loss / total_samples, "accuracy": correct_predictions / total_samples})

        epoch_loss = total_loss / total_samples
        epoch_accuracy = correct_predictions / total_samples
        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_accuracy)
        print(f"Epoch {epoch+1} Training Loss: {epoch_loss:.4f}, Training Accuracy: {epoch_accuracy:.4f}")
        
        
        # Optionally, checkpoint the model.
        if checkpoint_dir:
            path = os.path.join(checkpoint_dir, f"checkpoint_{epoch}.pt")
            torch.save(model.state_dict(), path)
    
    # Evaluation on the validation set
       # Evaluation loop
    model.eval()
    eval_loss = 0
    eval_correct_predictions = 0
    eval_total_samples = 0
    current_epoch_true_labels = []
    current_epoch_predicted_labels = []
    with torch.no_grad():
        progress_bar_eval = tqdm(eval_dataloader, desc=f"Evaluating Epoch {epoch+1}", unit="batch")
        for batch in progress_bar_eval:
            
            inputs = BaseDinov2.image_processor(images=list(batch[0]), return_tensors="pt", do_rescale=False).to(device)
            labels = batch[1].to(device)

            outputs = model(**inputs).logits
            loss = criterion(outputs, labels)

            eval_loss += loss.item() * inputs['pixel_values'].size(0)
            _, predicted = torch.max(outputs, 1)
            eval_correct_predictions += (predicted == labels).sum().item()
            eval_total_samples += labels.size(0)

            current_epoch_true_labels.extend(labels.cpu().numpy())
            current_epoch_predicted_labels.extend(predicted.cpu().numpy())
        

            progress_bar_eval.set_postfix({"loss": eval_loss / eval_total_samples, "accuracy": eval_correct_predictions / eval_total_samples})

    eval_epoch_loss = eval_loss / eval_total_samples
    eval_epoch_accuracy = eval_correct_predictions / eval_total_samples
    eval_losses.append(eval_epoch_loss)
    eval_accuracies.append(eval_epoch_accuracy)
    print(f"Epoch {epoch+1} Evaluation Loss: {eval_epoch_loss:.4f}, Evaluation Accuracy: {eval_epoch_accuracy:.4f}")
    
    all_true_labels.extend(current_epoch_true_labels)
    all_predicted_labels.extend(current_epoch_predicted_labels)
    
    # Report the metric to Ray Tune.
    tune.report({"loss": eval_epoch_loss})

In [64]:
asha_scheduler = ASHAScheduler(
    time_attr='training_iteration',
    metric='loss',
    mode='min',
    max_t=100,           # max training iterations per trial
    grace_period=10,     # min iterations before stopping
    reduction_factor=3,
    brackets=1,
)

optuna_search = OptunaSearch(metric="loss", mode="min", seed=42)

# define search space in a config dictionary, i.e what are the values you want to try, this is just example of format
'''
config = {
    "lr": tune.loguniform(1e-5, 1e-2),
    "weight_decay": tune.loguniform(1e-6, 1e-2),
    "dropout": tune.uniform(0.1, 0.5),
    "hidden_sizes": tune.choice([[256, 128], [512, 256], None]),
    "activation": tune.choice(["relu", "tanh"]),
    "freeze_backbone": tune.choice([True, False]),
    "num_epochs": 2, 
    "optimiser": tune.choice([optim.Adam, optim.SGD]),
    "criterion": tune.choice([nn.CrossEntropyLoss, nn.NLLLoss]),
}
'''

# this is what i specified for the example because i am running on cpu
config = {
    "lr": tune.loguniform(1e-5, 1e-2),
    "weight_decay": tune.loguniform(1e-6, 1e-2),
    "dropout": tune.uniform(0.1, 0.5),
    "hidden_sizes": tune.choice([[256, 128], [512, 256], None]),
    "activation":tune.choice(["leaky_relu","gelu"]),
    "freeze_backbone": tune.choice([True]),
    "num_epochs": 2,
    "optimiser": tune.choice([optim.AdamW, optim.SGD]),
    "criterion": tune.choice([nn.CrossEntropyLoss, nn.NLLLoss]),
}

# tuner object
tuner = tune.Tuner(
    tune.with_resources(train_model, {}), # specify based on the device u using because by default it uses all, i.e if u have 4 cpus; it does 4 concurrent trials
    tune_config=tune.TuneConfig(
        scheduler=asha_scheduler,
        search_alg=optuna_search,
        num_samples=2,  # number of hyperparameter combinations to try. Not the same as epochs
    ),
    run_config=tune.RunConfig(verbose=1),
    param_space=config,
)

results = tuner.fit()
print("Best config:", results.get_best_result(metric="loss", mode="min").config)

2025-04-07 15:14:51,742	ERROR tune_controller.py:1331 -- Trial task failed for trial train_model_7f3f5aec
Traceback (most recent call last):
  File "/opt/anaconda3/envs/rose/lib/python3.10/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/opt/anaconda3/envs/rose/lib/python3.10/site-packages/ray/_private/auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/opt/anaconda3/envs/rose/lib/python3.10/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/opt/anaconda3/envs/rose/lib/python3.10/site-packages/ray/_private/worker.py", line 2782, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, timeout=timeout)
  File "/opt/anaconda3/envs/rose/lib/python3.10/site-packages/ray/_private/worker.py", line 929, in get_objects
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(TypeError): ray::Imp

RuntimeError: No best trial found for the given metric: loss. This means that no trial has reported this metric, or all values reported for this metric are NaN. To not ignore NaN values, you can set the `filter_nan_and_inf` arg to False.